## Getting Started with MAF - Creating and Chatting with our Agent (At Microsoft Foundry Level)

![MAFAgent](./Assets/MAFAgent.png)

### Installing Required Libraries

In [ ]:
%pip install agent-framework==1.0.0b251209 python-dotenv azure-ai-projects==2.0.0b2

### Setting up the Environment

In [ ]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential

load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

### Creating the Foundry Client and Agent

In [ ]:
from agent_framework.azure import AzureAIClient
from azure.identity.aio import AzureCliCredential
from azure.ai.projects.aio import AIProjectClient

async def create_agent():
    credential = AzureCliCredential()
    
    # creating the Foundry Project Client
    project_client = AIProjectClient(
        endpoint=project_endpoint,
        credential=credential
    )

    # creating a conversation using the OpenAI Client
    openai_client = project_client.get_openai_client()
    conversation = await openai_client.conversations.create()
    conversation_id = conversation.id
    print("Conversation ID: ", conversation_id)
    
    # creating the Azure AI Client to interact with the Agent in Foundry
    agent_client = AzureAIClient(
        project_client = project_client,
        conversation_id = conversation_id,
        model_deployment_name=model
    )
    
    # creating an agent in Foundry
    agent = agent_client.create_agent(
        name="BatmanAgent",
        instructions="You are Batman, the dark knight of Gotham City."
    )
    return agent, credential, agent_client

agent, credential, agent_client = await create_agent()

### Chatting with the Agent - Non Streaming

In [ ]:
async def non_streaming_example():
    query = "Who is the Joker?"
    result = await agent.run(query)
    print("Agent Response:", result)

await non_streaming_example()

### Chatting with the Agent - Streaming

In [ ]:
# Running the agent with streaming responses
async def streaming_chat():
    async for update in agent.run_stream("Tell me something interesting about Gotham City in 10 points"):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming is complete

await streaming_chat()

### Giving our Agent an Image

![joker_interrogation_scene](./Assets/joker_interrogation_scene.png)

In [ ]:
from agent_framework import ChatMessage, TextContent, DataContent, Role

# Load image from local file
with open("./Assets/joker_interrogation_scene.png", "rb") as image_file:
    image_bytes = image_file.read()

# Create a message with text and image data
message = ChatMessage(
    role = Role.USER,
    contents = [
        TextContent(text = "Tell me about the incident in this image."),
        DataContent(
            data = image_bytes,
            media_type = "image/png"
        )
    ]
)

# Creating an async function for the entire image analysis run
async def analyze_image():
    response = await agent.run(message)
    print(f"Agent: {response.text}")

# invoke the async function
await analyze_image()